In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from seaborn import heatmap
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import plotly.express as px
from matplotlib import rcParams
import seaborn as sns
from sklearn.model_selection import KFold
%matplotlib inline

import random
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

# Data preprocessing
We use several methods to deal with the data, the first one is to drop some columns not appearing in all years.  
we have six years data of happiness in all contries in the world preprocess of the data:   
1. rename the columns that has the same meaning in all years
2. drop the uesless column or some columns only appears in just some years
3. fullfill the shortage data with mean value
4. put all trainning data in a same set

In [ ]:
df_2019=pd.read_csv('../input/world-happiness-report/2019.csv')
df_2018=pd.read_csv('../input/world-happiness-report/2018.csv')
df_2017=pd.read_csv('../input/world-happiness-report/2017.csv')
df_2016=pd.read_csv('../input/world-happiness-report/2016.csv')
df_2015=pd.read_csv('../input/world-happiness-report/2015.csv')

# pick up common features and rename

# print(df_2015.columns)
# print(df_2016.columns)
# print(df_2017.columns)
# print(df_2018.columns)
# print(df_2019.columns)

df_2017 = df_2017.rename(columns = {'Happiness.Score' : 'Happiness Score', 'Happiness.Rank' : 'Happiness Rank', 'Economy..GDP.per.Capita.' : 'Economy (GDP per Capita)', 'Health..Life.Expectancy.' : 'Health (Life Expectancy)','Trust..Government.Corruption.' : 'Trust (Government Corruption)', 'Dystopia.Residual' : 'Dystopia Residual'})
df_2018 = df_2018.rename(columns = {'Healthy life expectancy' : 'Health (Life Expectancy)', 'Social support' : 'Family', 'Score' : 'Happiness Score', 'Country or region':'Country', 'Overall rank' : 'Happiness Rank', 'GDP per capita' : 'Economy (GDP per Capita)', 'Perceptions of corruption' : 'Trust (Government Corruption)'})
df_2019 = df_2019.rename(columns = {'Freedom to make life choices' : 'Freedom', 'Score' : 'Happiness Score', 'Country or region':'Country', 'Overall rank' : 'Happiness Rank', 'GDP per capita' : 'Economy (GDP per Capita)', 'Social support' : 'Family','Healthy life expectancy' : 'Health (Life Expectancy)','Freedom to make life choices' : 'Freedom', 'Perceptions of corruption' : 'Trust (Government Corruption)'})

df_2015.drop(["Happiness Rank", "Standard Error", "Dystopia Residual"], inplace = True, axis = 1)
df_2016.drop(["Dystopia Residual", "Happiness Rank", "Lower Confidence Interval", "Upper Confidence Interval"], inplace = True, axis = 1)
df_2017.drop(["Happiness Rank", "Dystopia Residual", "Whisker.low", "Whisker.high"], inplace = True, axis = 1)
df_2018.drop(["Freedom to make life choices", "Happiness Rank"], inplace = True, axis = 1)
df_2019.drop("Happiness Rank", inplace = True, axis = 1)

#add country's region
country_rigion_map = pd.DataFrame({'Country': df_2015['Country'], 'Region': df_2015['Region']})
df_2017 = pd.merge(country_rigion_map, df_2017, how='right', on=['Country'])
df_2018 = pd.merge(country_rigion_map, df_2018, how='right', on=['Country'])
df_2019 = pd.merge(country_rigion_map, df_2019, how='right', on=['Country'])

df_2015['year'] = 2015
df_2016['year'] = 2016
df_2017['year'] = 2017
df_2018['year'] = 2018
df_2019['year'] = 2019

df_all = pd.concat([df_2019,df_2018,df_2017,df_2016,df_2015])
# fill null values with mean
df_all.fillna((df_all.mean()), inplace = True)
print(df_all.info())

df_all.to_csv('/kaggle/working/df_all.csv')

# analysis of the raw data


**the average score of each years**

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(df_all.groupby("year").mean()["Happiness Score"])
plt.ylabel("Happiness Score")
plt.ylim(5, 5.6)
plt.show()

**the top 10 countries which has the happinest value**

In [ ]:
df_top = df_all.head(10).sort_values('Happiness Score', ascending = True)

px.bar(df_top, x='Happiness Score', y='Country', title = "top 10")

**Visulaize the Variability**

In [ ]:
# Visulaize the Variability
plt.rcParams['figure.figsize'] = (10,10)
df_all[['Happiness Score',
       'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)',
       'Freedom', 'Generosity', 'Trust (Government Corruption)',]].hist();

df_all.columns

In [ ]:
plt.figure(figsize=(8,5))
col=df_all.corr()
sns.heatmap(col,annot=True,fmt='.2f');
plt.title("Correlation Map from Past 4 years from 2015 - 2019")
plt.show()

# baseline method
Here, we try to use constant value as a baseline

In [ ]:
# y = df_all['Happiness Score']
# X = df_all[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Generosity', 
#           'Trust (Government Corruption)']] 
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
y = df_all['Happiness Score']
X = pd.get_dummies(df_all.drop(["Happiness Score",'year','Country'], axis=1)).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0, shuffle=False)
final_mse = []
use_method = []
print('X shape', end="")
print(X_train.shape)
constant_mse = mean_squared_error(np.ones(y_test.shape) * np.mean(y_train), y_test)
final_mse.append(constant_mse)
use_method.append('baseline')
print("r2 score is %.2f" % r2_score(y_test, np.ones(y_test.shape) * np.mean(y_train)))
print("Constant MSE: %.2f" % constant_mse)

# KNN

In [ ]:
plt.figure(figsize=(5,5))
mse_list = []
k_range = range(1, 51)
for k in k_range:
    model = KNeighborsRegressor(k)
    model.fit(X_train, y_train)
    mse_list.append(mean_squared_error(model.predict(X_test), y_test))
k_list = [k for k in k_range]
plt.plot(k_list, mse_list, label = "mse")
plt.plot(k_list, [constant_mse for i in range(len(k_list))], label="baseline")
plt.legend()
plt.title("use KNN")
plt.show()
final_mse.append(min(mse_list))
use_method.append('KNN')

# best parameter for knn
knnReg = KNeighborsRegressor (n_neighbors = 4)
knnReg.fit(X_train,y_train)
y_pred = knnReg.predict(X_test).reshape(-1, 1)

score = r2_score(y_test,y_pred)

# lasso regression
Add some description

In [ ]:
y = df_all['Happiness Score']
X = pd.get_dummies(df_all.drop(["Happiness Score",'year','Country'], axis=1)).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0, shuffle=False)
# X_train = pd.get_dummies(df_all.drop(["Happiness Score",'year','Country'], axis=1)).values
# X_test = pd.get_dummies(df_2020.drop(["Happiness Score",'year','Country'], axis=1)).values
# y_train = df_all['Happiness Score']
# y_test = df_2020['Happiness Score']


lasso = Pipeline(steps = [
    ('ss', StandardScaler()),
    ('poly', PolynomialFeatures(1)),
    ('sgd',  SGDRegressor(penalty="l1", alpha=0.1,random_state=0)),
])

lasso.fit(X_train,y_train)
print("mse:", end="")
print(mean_squared_error(lasso.predict(X_test), y_test))
print("r2 score:", end="")
print(r2_score(y_test, lasso.predict(X_test)))
final_mse.append(mean_squared_error(lasso.predict(X_test), y_test))
use_method.append('lasso')
def find_zero_and_nonezero_index(coefs):
    nonzero_index = []
    zero_index = []
    for i in range(len(coefs)-1):
        if coefs[i] != 0:
            nonzero_index.append(i)
        else:
            zero_index.append(i)
    return nonzero_index,zero_index

nonzero_index, zero_index = find_zero_and_nonezero_index(lasso['sgd'].coef_)
print("find non zero index and zero index", end=" ")
print(nonzero_index, zero_index)

### after delete the inrelevant features

In [ ]:
n_X_train = X_train[:,nonzero_index].reshape(-1,8)
n_X_test = X_test[:,nonzero_index].reshape(-1,8)

lasso.fit(n_X_train,y_train)
print("mse:", end="")
print(mean_squared_error(lasso.predict(n_X_test),y_test))
print("r2 score:", end="")
print(r2_score(y_test, lasso.predict(n_X_test)))
plt.figure(figsize=(5,5))
plt.bar(['before','after'],[final_mse[-1],mean_squared_error(lasso.predict(n_X_test),y_test)])


It results that all the data are relevent with the happiness 

# Ridge regression


In [ ]:
ridge = pipeline = Pipeline([
    ('ss', StandardScaler()),
    ('poly', PolynomialFeatures(1)),
    ('sgd',  SGDRegressor(penalty="l2",alpha=1,random_state=0)),
])
ridge.fit(X_train, y_train)
print("mse:", end="")
print(mean_squared_error(ridge.predict(X_test), y_test))
print("r2 score:", end="")
print(r2_score(y_test, ridge.predict(X_test)))
final_mse.append(mean_squared_error(ridge.predict(X_test), y_test))
use_method.append('ridge')

# Linear Regression

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
print("mse:", end=" ")
print(mean_squared_error(model.predict(X_test), y_test))
print("r2 score:", end=" ")
print(r2_score(y_test, model.predict(X_test)))
print(model.score(X_test, y_test))
final_mse.append(mean_squared_error(model.predict(X_test), y_test))
use_method.append('linear')

# random forest

In [ ]:
res = []
x_axis = []
for i in range(1,30):
    rfrg = RandomForestRegressor(n_estimators = 100,max_depth=i, random_state=0)
    rfrg.fit(X_train, y_train)
    res.append(mean_squared_error(rfrg.predict(X_test), y_test))
    x_axis.append(i)

plt.figure(figsize=(5,5))
plt.plot(x_axis,res)
plt.plot(x_axis, [constant_mse for i in range(len(res))], label="baseline")
plt.legend()
plt.title("use random forest")
plt.show()
print("mes: ", end="")
print(min(res))
final_mse.append(min(res))
use_method.append('random_forest')

In [ ]:
plt.figure(figsize=(8,5))
plt.bar(['Economy', 'Family', 'Health ', 'Freedom', 'Generosity', 
          'Government Trust'], rfrg.feature_importances_[:6])


also use decision tree we can know wich feature influnences the happiness most

# GradientBoostingRegressor
use subsample to lead to a reduction of variance and an increase in bias

In [ ]:
#reg = GradientBoostingRegressor(random_state=0)
#reg.fit(X_train, y_train)
res = []
x_axis = []
learning_rate = 0.01
while learning_rate <= 0.2:
    reg = GradientBoostingRegressor(random_state=0,subsample=0.7,learning_rate = learning_rate)
    reg.fit(X_train, y_train)
    res.append(mean_squared_error(reg.predict(X_test), y_test))
    x_axis.append(learning_rate)
    learning_rate+= 0.01
    
plt.figure(figsize=(5,5))
plt.plot(x_axis,res,label="boosting")
plt.plot(x_axis, [constant_mse for i in range(len(res))], label="baseline")
plt.legend()
plt.title("use boosting")
plt.show()
print(min(res),x_axis[res.index(min(res))])
final_mse.append(min(res))
use_method.append('boosting')
# print("mse: ", end=" ")
# print(mean_squared_error(regr.predict(X_test), y_test))
# print("r2 score:", end=" ")
# print(r2_score(y_test, regr.predict(X_test)))

**from now on, we conclude all the methods we used and the result mse we get from the best parameters we tried**

In [ ]:
plt.figure(figsize=(5,5))
plt.bar(use_method,final_mse)
plt.title("result")
for i in range(len(use_method)):
    plt.text(use_method[i] , final_mse[i],round(final_mse[i],3),color='red', fontweight='bold',horizontalalignment = 'center')
plt.show()

divede the date into different rigions

In [ ]:
mse_list_r = []
regions = ['Australia and New Zealand','Central and Eastern Europe','Eastern Asia','Latin America and Caribbean','Middle East and Northern Africa',
              'North America','Southeastern Asia','Southern Asia','Sub-Saharan Africa','Western Europe']
for region in regions:
    df_all_region = df_all[df_all['Region'] == region]
    y_region = df_all_region['Happiness Score']
    X_region = pd.get_dummies(df_all_region.drop(["Happiness Score",'year','Country'], axis=1)).values

    r_X_train, r_X_test, r_y_train, r_y_test = train_test_split(X_region, y_region, test_size=0.20, random_state=0, shuffle=False)
    reg = GradientBoostingRegressor(random_state=0,subsample=0.7,learning_rate = 0.1)
    reg.fit(r_X_train, r_y_train)
    mse_list_r.append(mean_squared_error(reg.predict(r_X_test), r_y_test))



regions_abre = ['au&nz','EU','EA','la&c','me&NA','NA','SAa','SAf','SS','WE']
plt.figure(figsize=(5,5))
plt.bar(regions_abre,mse_list_r)
plt.title("result")
for i in range(len(regions)):
    plt.text(regions_abre[i] , mse_list_r[i],round(mse_list_r[i],3),color='red', fontweight='bold',horizontalalignment = 'center')
plt.show()




# Neural network
## try neural network here to deal with data from 2016 - 2019 all

In [ ]:
y = df_all['Happiness Score']
X = df_all[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Generosity', 
          'Trust (Government Corruption)']] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True)

In [ ]:
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)
print(X_train.shape)

## Let's first try to use a simple neural network

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, net_in):
        super(Net, self).__init__()
        self.net=torch.nn.Sequential(
            torch.nn.Linear(net_in,5),
            torch.nn.SiLU(),
            torch.nn.Linear(5, 1),
        )
        
    def forward(self, x):
        return self.net(x)

net = Net(X_train.shape[1])
optim=torch.optim.Adam(Net.parameters(net),lr=0.001)
Loss=torch.nn.MSELoss()

epoch_all = 40000
len1 = len(X_train)
test_error = []
r2_score_test = []
train_error = []
r2_score_train = []
count = []
prev = 0.0
current = 0.0
for epoch in range(epoch_all):
    loss=None
    for i in range(100):
        index = random.randint(0, len1 - 1)
        y_predict=net(X_train[index])
        loss=Loss(y_predict,y_train[index])
        optim.zero_grad()
        loss.backward()
        optim.step()
    if (epoch + 1) % 100 == 0:
        prev = current
        current = mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())
        if abs(prev - current) < 1e-6:
            break
        train_error.append(mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy()))
        test_error.append(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
        r2_score_test.append(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))
        r2_score_train.append(r2_score(y_train.data.numpy(), net(X_train).data.numpy()))
        count.append(epoch)
        if (epoch + 1) % 2000 == 0:
            print("step: {0} , loss: {1}".format(epoch+1,mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())))

plt.figure(figsize=(5,5))
plt.plot(count, train_error, label = "train")
plt.plot(count, test_error, label="test")
plt.title("MSE for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()

plt.figure(figsize=(5,5))
plt.plot(count, r2_score_train, label = "train")
plt.plot(count, r2_score_test, label="test")
plt.title("r2 score for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()

print(net)
print("mse: ", end=" ")
print(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
print("r2 score = ", end= " ")
print(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))

print("the best score of neural network is :")
print("r2 score = ", end= " ")
print(max(r2_score_test))
print("mse: ", end=" ")
print(min(test_error))

## Then let's try the neural network with more nodes

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, net_in):
        super(Net, self).__init__()
        self.net=torch.nn.Sequential(
            torch.nn.Linear(net_in,5),
            torch.nn.SiLU(),
            torch.nn.Linear(5, 3),
            torch.nn.SiLU(),
            torch.nn.Linear(3, 1)
        )
        
    def forward(self, x):
        return self.net(x)

net = Net(X_train.shape[1])
optim=torch.optim.Adam(Net.parameters(net),lr=0.001)
Loss=torch.nn.MSELoss()

epoch_all = 40000
len1 = len(X_train)
test_error = []
r2_score_test = []
train_error = []
r2_score_train = []
count = []
prev = 0.0
current = 0.0
for epoch in range(epoch_all):
    loss=None
    for i in range(100):
        index = random.randint(0, len1 - 1)
        y_predict=net(X_train[index])
        loss=Loss(y_predict,y_train[index])
        optim.zero_grad()
        loss.backward()
        optim.step()
    if (epoch + 1) % 100 == 0:
        prev = current
        current = mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())
        if abs(prev - current) < 1e-6:
            break
        train_error.append(mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy()))
        test_error.append(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
        r2_score_test.append(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))
        r2_score_train.append(r2_score(y_train.data.numpy(), net(X_train).data.numpy()))
        count.append(epoch)
        if (epoch + 1) % 2000 == 0:
            print("step: {0} , loss: {1}".format(epoch+1,mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())))

plt.figure(figsize=(5,5))
plt.plot(count, train_error, label = "train")
plt.plot(count, test_error, label="test")
plt.title("MSE for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()

plt.figure(figsize=(5,5))
plt.plot(count, r2_score_train, label = "train")
plt.plot(count, r2_score_test, label="test")
plt.title("r2 score for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()

print(net)
print("mse: ", end=" ")
print(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
print("r2 score = ", end= " ")
print(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))

print("the best score of neural network is :")
print("r2 score = ", end= " ")
print(max(r2_score_test))
print("mse: ", end=" ")
print(min(test_error))

## Let's try to fit the data with more neurons then previous one 

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, net_in):
        super(Net, self).__init__()
        self.net=torch.nn.Sequential(
            torch.nn.Linear(net_in,10),
            torch.nn.SiLU(),
            torch.nn.Linear(10, 10),
            torch.nn.SiLU(),
            torch.nn.Linear(10, 1)
        )
        
    def forward(self, x):
        return self.net(x)

net = Net(X_train.shape[1])
optim=torch.optim.Adam(Net.parameters(net),lr=0.0001)
Loss=torch.nn.MSELoss()

epoch_all = 40000
len1 = len(X_train)
test_error = []
r2_score_test = []
train_error = []
r2_score_train = []
count = []
prev = 0.0
current = 0.0
for epoch in range(epoch_all):
    loss=None
    for i in range(100):
        index = random.randint(0, len1 - 1)
        y_predict=net(X_train[index])
        loss=Loss(y_predict,y_train[index])
        optim.zero_grad()
        loss.backward()
        optim.step()
    if (epoch + 1) % 100 == 0:
        prev = current
        current = mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())
        if abs(prev - current) < 1e-6:
            break
        train_error.append(mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy()))
        test_error.append(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
        r2_score_test.append(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))
        r2_score_train.append(r2_score(y_train.data.numpy(), net(X_train).data.numpy()))
        count.append(epoch)
        if (epoch + 1) % 2000 == 0:
            print("step: {0} , loss: {1}".format(epoch+1,mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())))

            

plt.figure(figsize=(5,5))
plt.plot(count, train_error, label = "train")
plt.plot(count, test_error, label="test")
plt.title("MSE for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()


plt.figure(figsize=(5,5))
plt.plot(count, r2_score_train, label = "train")
plt.plot(count, r2_score_test, label="test")
plt.title("r2 score for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()


print(net)
print("mse: ", end=" ")
print(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
print("r2 score = ", end= " ")
print(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))

print("the best score of neural network is :")
print("r2 score = ", end= " ")
print(max(r2_score_test))
print("mse: ", end=" ")
print(min(test_error))

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, net_in):
        super(Net, self).__init__()
        self.net=torch.nn.Sequential(
            torch.nn.Linear(net_in,30),
            torch.nn.SiLU(),
            torch.nn.Linear(30, 30),
            torch.nn.SiLU(),
            torch.nn.Linear(30, 1)
        )
        
    def forward(self, x):
        return self.net(x)

net = Net(X_train.shape[1])
optim=torch.optim.Adam(Net.parameters(net),lr=0.0001)
Loss=torch.nn.MSELoss()

epoch_all = 40000
len1 = len(X_train)
test_error = []
r2_score_test = []
train_error = []
r2_score_train = []
count = []
prev = 0.0
current = 0.0
for epoch in range(epoch_all):
    loss=None
    for i in range(100):
        index = random.randint(0, len1 - 1)
        y_predict=net(X_train[index])
        loss=Loss(y_predict,y_train[index])
        optim.zero_grad()
        loss.backward()
        optim.step()
    if (epoch + 1) % 100 == 0:
        prev = current
        current = mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())
        if abs(prev - current) < 1e-5:
            break
        train_error.append(mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy()))
        test_error.append(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
        r2_score_test.append(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))
        r2_score_train.append(r2_score(y_train.data.numpy(), net(X_train).data.numpy()))
        count.append(epoch)
        if (epoch + 1) % 2000 == 0:
            print("step: {0} , loss: {1}".format(epoch+1,mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())))

            

plt.figure(figsize=(5,5))
plt.plot(count, train_error, label = "train")
plt.plot(count, test_error, label="test")
plt.title("MSE for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()


plt.figure(figsize=(5,5))
plt.plot(count, r2_score_train, label = "train")
plt.plot(count, r2_score_test, label="test")
plt.title("r2 score for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()


print(net)
print("mse: ", end=" ")
print(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
print("r2 score = ", end= " ")
print(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))

print("the best score of neural network is :")
print("r2 score = ", end= " ")
print(max(r2_score_test))
print("mse: ", end=" ")
print(min(test_error))

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, net_in):
        super(Net, self).__init__()
        self.net=torch.nn.Sequential(
            torch.nn.Linear(net_in,50),
            torch.nn.SiLU(),
            torch.nn.Linear(50, 50),
            torch.nn.SiLU(),
            torch.nn.Linear(50, 1)
        )
        
    def forward(self, x):
        return self.net(x)

net = Net(X_train.shape[1])
optim=torch.optim.Adam(Net.parameters(net),lr=0.0001)
Loss=torch.nn.MSELoss()

epoch_all = 40000
len1 = len(X_train)
test_error = []
r2_score_test = []
train_error = []
r2_score_train = []
count = []
prev = 0.0
current = 0.0
for epoch in range(epoch_all):
    loss=None
    for i in range(100):
        index = random.randint(0, len1 - 1)
        y_predict=net(X_train[index])
        loss=Loss(y_predict,y_train[index])
        optim.zero_grad()
        loss.backward()
        optim.step()
    if (epoch + 1) % 100 == 0:
        prev = current
        current = mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())
        if abs(prev - current) < 1e-5:
            break
        train_error.append(mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy()))
        test_error.append(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
        r2_score_test.append(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))
        r2_score_train.append(r2_score(y_train.data.numpy(), net(X_train).data.numpy()))
        count.append(epoch)
        if (epoch + 1) % 2000 == 0:
            print("step: {0} , loss: {1}".format(epoch+1,mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())))

            

plt.figure(figsize=(5,5))
plt.plot(count, train_error, label = "train")
plt.plot(count, test_error, label="test")
plt.title("MSE for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()


plt.figure(figsize=(5,5))
plt.plot(count, r2_score_train, label = "train")
plt.plot(count, r2_score_test, label="test")
plt.title("r2 score for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()


print(net)
print("mse: ", end=" ")
print(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
print("r2 score = ", end= " ")
print(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))

print("the best score of neural network is :")
print("r2 score = ", end= " ")
print(max(r2_score_test))
print("mse: ", end=" ")
print(min(test_error))

## Try to use few layers to eliminate the problem of overfitting

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, net_in):
        super(Net, self).__init__()
        self.net=torch.nn.Sequential(
            torch.nn.Linear(net_in,30),
            torch.nn.Tanh(),
            torch.nn.Linear(30, 1),
        )
        
    def forward(self, x):
        return self.net(x)

net = Net(X_train.shape[1])
optim=torch.optim.Adam(Net.parameters(net),lr=0.0001)
Loss=torch.nn.MSELoss()

epoch_all = 40000
len1 = len(X_train)
test_error = []
r2_score_test = []
train_error = []
r2_score_train = []
count = []
prev = 0.0
current = 0.0
for epoch in range(epoch_all):
    loss=None
    for i in range(100):
        index = random.randint(0, len1 - 1)
        y_predict=net(X_train[index])
        loss=Loss(y_predict,y_train[index])
        optim.zero_grad()
        loss.backward()
        optim.step()
    if (epoch + 1) % 100 == 0:
        prev = current
        current = mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())
        if abs(prev - current) < 1e-6:
            break
        train_error.append(mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy()))
        test_error.append(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
        r2_score_test.append(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))
        r2_score_train.append(r2_score(y_train.data.numpy(), net(X_train).data.numpy()))
        count.append(epoch)
        if (epoch + 1) % 2000 == 0:
            print("step: {0} , loss: {1}".format(epoch+1,mean_squared_error(net(X_train).data.numpy(), y_train.data.numpy())))


plt.figure(figsize=(5,5))
plt.plot(count, train_error, label = "train")
plt.plot(count, test_error, label="test")
plt.title("MSE for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()


plt.figure(figsize=(5,5))
plt.plot(count, r2_score_train, label = "train")
plt.plot(count, r2_score_test, label="test")
plt.title("r2 score for train and test")
plt.ylim((0,1))
plt.legend()
plt.show()

print("mse: ", end=" ")
print(mean_squared_error(net(X_test).data.numpy(), y_test.data.numpy()))
print("r2 score = ", end= " ")
print(r2_score(y_test.data.numpy(), net(X_test).data.numpy()))

print("the best score of neural network is :")
print("r2 score = ", end= " ")
print(max(r2_score_test))
print("mse: ", end=" ")
print(min(test_error))

As we can see, due to the size and dimension of the data, it's easy to overfit the model.   
The best model for using neural network is 
```
Net(
  (net): Sequential(
    (0): Linear(in_features=6, 
        out_features=5, bias=True)
    (1): SiLU()
    (2): Linear(in_features=5, 
        out_features=1, bias=True)
  )
)
```